<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Step11" data-toc-modified-id="Step11-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Step11</a></span></li><li><span><a href="#Step12" data-toc-modified-id="Step12-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Step12</a></span></li><li><span><a href="#Step13" data-toc-modified-id="Step13-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Step13</a></span></li><li><span><a href="#Step14" data-toc-modified-id="Step14-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step14</a></span></li><li><span><a href="#Step15,16" data-toc-modified-id="Step15,16-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step15,16</a></span></li></ul></div>

## Step11

In [3]:
import numpy as np

# 변수 클래스
class Variable:
    def __init__(self, data):
        # 입력 dtype 확인
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} type is not supported.")
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.creator is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator] # 현재 입력 변수를 만든 창조자 함수 가져오기
        while funcs:
            f = funcs.pop()             # 1.현재 입력 변수를 만든 창조자 함수 가져오기
            x, y = f.input, f.output    # 2.창조함수의 입력,출력 변수 가져오기
            x.grad = f.backward(y.grad) # 3.창조함수의 역전파 메소드 호출
            
            if x.creator is not None:
                funcs.append(x.creator)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
# 공통함수 클래스
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        
        return output
    
    def forward(self, x):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gy):
        raise NotImplementedError("This method should be called in other function class")

In [41]:
# 변수 담는 클래스
class Variable:
    def __init__(self, data):
        # 입력변수 dtype 유효성 검증
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported!")
                
        self.data = data
        self.grad = None  # 해당변수의 기울기값
        self.creator = None # 해당변수를 만들어낸 창조함수 기록
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        # 순전파 후, 특정 변수의 기울기값이 None이라는 것은 최종 출력값에 위치하고있음을 의미
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        # 역전파 수행하기 위해서 해당 변수를 만든 창조자 함수를 가져오기
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output  # 입력,출력 변수 가져오기 for 미분계산
            x.grad = f.backward(y.grad) # 여기서 y.grad = self.grad 임
            
            # 지금 가져온 해당 변수를 만든 함수의 입력 변수에 창조자 함수가 없다는 것은 입력층까지 모두 다다랐다는 것을 의미
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [24]:
# 공통 함수 클래스 -> 입/출력을 여러개 받을 수 있도록 하기
class Function:
    def __call__(self, inputs: list):
        """
        Args:
            inputs: [Variable(..), Variable(..), ...]
        """
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs
        
    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")

In [25]:
# Add 함수 클래스 만들어보기
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)  # 출력은 튜플 형태로!
    
def add(xs):
    return Add()(xs) 

In [29]:
xs = [Variable(np.array(2)), Variable(np.array(5))]
ys = add(xs)
y = ys[0]
y.data

array(7)

In [45]:
def func(*inputs):
    res = [x for x in inputs]
    print(res)
    
func(1, 2,3)

[1, 2, 3]


## Step12

In [46]:
# 사용자 편의를 개선한 Function 클래스
class Function:
    def __call__(self, *inputs):
        print('type:', type(inputs[0]))
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        # 각 출력에 창조자 함수 설정
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        # 출력값 원소가 하나면 하나만을 반환
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")
    
    
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)
    
def add(*xs):
    return Add()(*xs)

In [48]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(5))

ys = add(x0, x1)
print(type(ys), ys)
ys.data

type: <class '__main__.Variable'>
<class '__main__.Variable'> <__main__.Variable object at 0x7fbb8aa38150>


array(7)

In [52]:
# 변수 담는 클래스
class Variable:
    def __init__(self, data):
        # 입력변수 dtype 유효성 검증
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported!")
                
        self.data = data
        self.grad = None  # 해당변수의 기울기값
        self.creator = None # 해당변수를 만들어낸 창조함수 기록
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        # 순전파 후, 특정 변수의 기울기값이 None이라는 것은 최종 출력값에 위치하고있음을 의미
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        # 역전파 수행하기 위해서 해당 변수를 만든 창조자 함수를 가져오기
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output  # 입력,출력 변수 가져오기 for 미분계산
            x.grad = f.backward(y.grad) # 여기서 y.grad = self.grad 임
            
            # 지금 가져온 해당 변수를 만든 함수의 입력 변수에 창조자 함수가 없다는 것은 입력층까지 모두 다다랐다는 것을 의미
            if x.creator is not None:
                funcs.append(x.creator)
                
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

# 이어서 개발자 편의를 위해 개선한 Function 클래스
class Function:
    def __call__(self, *inputs):
        """
        Args:
            inputs: [Variable(..), Variable(..), ...]
        """
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    

def add(x0, x1):
    return Add()(x0, x1)

In [54]:
x0 = Variable(np.array(1))
x1 = Variable(np.array(10))
y = add(x0, x1)
print(type(y), y)
print(y.data)

<class '__main__.Variable'> <__main__.Variable object at 0x7fbb8aadb910>
11


## Step13

In [55]:
# 이제 역전파를 구현할 때 가변길이 인수를 받을 수 있도록 대응하기 -> Variable 클래스를 구현

# 기존 클래스
class Variable:
    def __init__(self, data):
        if self.data is not None:
            if not isinstance(self.data, np.ndarray):
                raise TypeError(f"{type(self.data)} dtype is not supported!")
                
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [63]:
# 변경된 클래스
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported!")
                
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            # 여러개 출력의 기울기값을 리스트에 담기
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) # 역전파 기울기 계산
            # 계산되어 반환된 기울기값을 튜플로 변환
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            # 각 변수에 대응하는 미분값 갱신
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)

In [66]:
# 함수 클래스
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        # 출력변수를 만든 창조자 함수 기록
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")
        
# 제곱함수 클래스
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
# 덧셈 클래스
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy
    
# PythonAPI로 변경
def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

In [72]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

print(z.grad)
print(z.data)
print(x.grad)
print(y.grad)

1.0
13.0
4.0
6.0


## Step14

In [77]:
# 같은 변수를 반복해서 사용해 미분하면 계산 결과에서 오차 발생하는 문제
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported!")
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            # 각 x변수에 기울기 매핑
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    funcs.append(x.creator)
                    
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        # 각 출력에 창조자 함수 설정
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")
        

class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
    
class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1
    
    def backward(self, gy):
        return gy, gy
    

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [80]:
x = Variable(np.array(3.0))
y = add(x, x)

print(y.data)
y.backward()
print(y.grad)
print(x.grad)

6.0
1.0
2.0


In [81]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)

print(y.data)
y.backward()
print(y.grad)
print(x.grad)

9.0
1.0
3.0


In [83]:
# 이번엔 동일한 변수가 서로 다른 미분계산을 수행할때, 즉, 서로 다른 미분 계산에 한쪽의 미분 계산값이 사용되어서 잘못된 결과를 초래
# 미분값을 초기화하는 메서드를 추가

class Variable:
    def __init__(self, data):
        if data is None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported")
        
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def cleargrad(self):
        self.grad = None
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
                    

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError("This method should be called in other function class")
        
    def backward(self, gys):
        raise NotImplementedError("This method should be called in other function class")
        

class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
    
class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1
    
    def backward(self, gy):
        return gy, gy
    

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [85]:
x = Variable(np.array(3.0))
y = add(x, x)
print(y.data)
y.backward()
print(y.grad)
print(x.grad)
print('-'*50)
y = add(add(x, x), x)
print(y.data)
y.backward()
print(y.grad)
print(x.grad)

6.0
1.0
2.0
--------------------------------------------------
9.0
1.0
5.0


In [86]:
# 미분 재설정 메소드 추가
x = Variable(np.array(3.0))
y = add(x, x)
print(y.data)
y.backward()
print(y.grad)
print(x.grad)
print('-'*50)
x.cleargrad()
y = add(add(x, x), x)
print(y.data)
y.backward()
print(y.grad)
print(x.grad)

6.0
1.0
2.0
--------------------------------------------------
9.0
1.0
3.0


## Step15,16

In [88]:
funcs = []
seen_set = set()

func1 = Function()
func2 = Function()
func3 = Function()

creators = [func1, func2, func3]

for f in creators:
    if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        print(funcs)
        print(seen_set)

{<__main__.Function object at 0x7fbb8abe2710>}
[<__main__.Function object at 0x7fbb8abe2710>, <__main__.Function object at 0x7fbb8abe2f50>]
{<__main__.Function object at 0x7fbb8abe2710>, <__main__.Function object at 0x7fbb8abe2f50>}
[<__main__.Function object at 0x7fbb8abe2710>, <__main__.Function object at 0x7fbb8abe2f50>, <__main__.Function object at 0x7fbb8abe2510>]
{<__main__.Function object at 0x7fbb8abe2510>, <__main__.Function object at 0x7fbb8abe2710>, <__main__.Function object at 0x7fbb8abe2f50>}


In [90]:
# 세대를 추가해서 역전파 시 함수 간의 우선순위 설정
# 함수와 그 함수에 들어간 입력변수는 같은 세대로!

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} dtype is not supported!")
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1  # 함수에 새겨진 세대값 + 1 
        
    def cleargrad(self):
        self.grad = None
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = []
        seen_sets = set()
        
        # 중첩함수 사용 조건: 1.감싸는 메서드에서만 사용 2.감싸는 메서드에 정의된 변수를 사용해야만 할 때
        def add_func(f):
            if f not in seen_sets:
                funcs.append(f)
                seen_sets.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        # 가장 먼저 출력층의 창조함수 add 
        add_func(self.creator)
        
        while funcs:
            f = funcs.pop()
            gys = [output.grad in output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            # 각 변수에 각 grad 갱신
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)